<a href="https://colab.research.google.com/github/Daivar/Deep_Learning_Models/blob/main/Fastai%2C_tabulae%2C_mebedings%2C_adult_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.tabular import *

df = pd.DataFrame({'col1': ['02/03/2017', '02/04/2017', '02/05/2017'], 'col2': ['a', 'b', 'a']})
df

In [ ]:
add_datepart(df, 'col1') # inplace
df.head()

### Splitai (continues categorical split)

In [ ]:
df = pd.DataFrame({'col1': [1.0, 2.0, 3.0], 'col2': ['a', 'b', 'a'], 'col3': [0.5, 1.2, 7.5], 'col4': ['ab', 'o', 'o']})
df

In [ ]:
cont_list, cat_list = cont_cat_split(df=df, max_card=20, dep_var='col4')
cont_list, cat_list

### Fast.ai tabular

In [ ]:
from fastai.tabular import *

!rm -rf ./sample_data/ ./data/

path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
df

In [ ]:
df.head

In [ ]:
df.dtypes

In [ ]:
dep_var = 'salary'

# separate all the variables that will need to be encoded
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']

# continuous variables will not need to be encoded
cont_names = ['age', 'fnlwgt', 'education-num']

# define the procedures we will do
procs = [FillMissing, Categorify]


In [ ]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

# path is just the working directory where temporary files/models will be saved.
# ... ref: https://forums.fast.ai/t/looking-for-documentation-on-path-argument-in-tabulardatabunch-from-df/34999/2

In [ ]:
test

In [ ]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [ ]:
data.show_batch(rows=100)

In [ ]:
compare
list(df.head(100)['occupation'])
data.show_batch


In [ ]:
learn = tabular_learner(data, layers=[5, 5], metrics=accuracy)

In [ ]:
learn = tabular_learner(data, layers=[5, 5, 5], metrics=accuracy)

In [ ]:
# fit(number_of_epochs, learning_rate)
learn.fit(5, 1e-2)

In [ ]:
# vizualize the model
learn.model

### 6 embedings were created for our categorical features

### Random Forest

In [ ]:
import pandas as pd
from fastai.tabular import *
from sklearn.ensemble import RandomForestClassifier

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
len(np.unique(df['education']))

In [ ]:
print(np.unique(df['education']))

In [ ]:
df.isnull().sum()

In [ ]:
df['missing_education-num'] = df['education-num'].isnull().map({True: 1, False: 0})
df['missing_occupation'] = df['occupation'].isnull().map({True: 1, False: 0})
df.head()

In [ ]:
def split_vals(a,n): 
  return a[:n].copy(), a[n:].copy()

In [ ]:
X = df.drop('salary', axis = 1)
y = df['salary']

In [ ]:
len(df)

In [ ]:
n_valid = int(len(df) * 0.2)  # same as Kaggle's test set size
n_trn = len(df)-n_valid

X_train, X_valid = split_vals(X, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, max_features=0.5, n_estimators=20)
clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_valid)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, predictions)